In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import PIL
import numpy as np
import random
from feret_utils import get_feret_files_and_tags_dict, run_face_detection, create_dataset, create_dataset_gs
from matplotlib.pyplot import imshow
from IPython.display import Image
import cv2 as cv
import tensorflow as tf

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Start by getting a dictionary holding file names of images as keys and id of person as value
feret_dict = get_feret_files_and_tags_dict()

In [3]:
# During training we want to see the affect of not using the "pr" and "pl" examples
list_of_excluded = []

In [44]:
# Create a datast, using only images which were detected in run_face_detection run, and crop the images to new size
subject_list, data_dict, mean_image, std_image = create_dataset_gs(feret_dict, subject_count=700, flip=True)

Info: subject reached No.0
Info: subject reached No.5
Info: subject reached No.10
Info: subject reached No.15
Info: subject reached No.20
Info: subject reached No.25
Info: subject reached No.30
Info: subject reached No.35
Info: subject reached No.40
Info: subject reached No.45
Info: subject reached No.50
Info: subject reached No.55
Info: subject reached No.60
Info: subject reached No.65
Info: subject reached No.70
Info: subject reached No.75
Info: subject reached No.80
Info: subject reached No.85
Info: subject reached No.90
Info: subject reached No.95
Info: subject reached No.100
Info: subject reached No.105
Info: subject reached No.110
Info: subject reached No.115
Info: subject reached No.120
Info: subject reached No.125
Info: subject reached No.130
Info: subject reached No.135
Info: subject reached No.140
Info: subject reached No.145
Info: subject reached No.150
Info: subject reached No.155
Info: subject reached No.160
Info: subject reached No.165
Info: subject reached No.170
Info: s

In [8]:
import pickle
# Save data_dict so we can unpickle it when needed
data_to_pickle = (subject_list, data_dict, mean_image, std_image)
with open('feret_data_dict_gs.pickle', 'wb') as handle:
    pickle.dump(data_to_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
import pickle
# Unpickle data_dict
with open('feret_data_dict_gs.pickle', 'rb') as handle:
    subject_list, data_dict, mean_image, std_image = pickle.load(handle)

In [5]:
print(data_dict['X_train'].shape)
print(data_dict['y_train'].shape)
print(data_dict['X_eval'].shape)
print(data_dict['y_eval'].shape)

(6931, 96, 96, 1)
(6931,)
(268, 96, 96, 1)
(268,)


In [7]:
ex_list = [119, 120, 167, 168, 193, 194, 253, 254, 341, 342, 515, 516, 547, 548, 669, 670, 775, 776, 1171, 1172,
 1181, 1182, 1213, 1214, 1275, 1276, 1395, 1396, 1517, 1518, 1521, 1522, 1531, 1532, 1613, 1614, 1685, 1686,
 1695, 1696, 1701, 1702, 1703, 1704, 1725, 1726, 1747, 1748, 1779, 1780, 1789, 1790, 1899, 1900, 1937, 1938,
 1945, 1946, 1951, 1952, 1953, 1954, 1955, 1956, 1987, 1988, 2093, 2094, 2107, 2108, 2115, 2116, 2125, 2126,
 2163, 2164, 2181, 2182, 2199, 2200, 2213, 2214, 2229, 2230, 2255, 2256, 2281, 2281, 2283, 2284, 2311, 2312,
 2333, 2334, 2381, 2382, 2397, 2398, 2411, 2412, 2497, 2498, 2519, 2520, 2673, 2674, 2683, 2684, 2743, 2744,
 2761, 2762, 2777, 2778, 2833, 2834, 2857, 2858, 2873, 2874, 2875, 2876, 2919, 2920, 2939, 2940, 3021, 3022,
 3067, 3068, 3105, 3106, 3143, 3144, 3183, 3184, 3213, 3214, 3227, 3228, 3233, 3234, 3245, 3246, 3251, 3252,
 3252, 3254, 3267, 3268, 3353, 3354, 3401, 3402, 3405, 3406, 3407, 3408, 3417, 3418, 3447, 3448, 3717, 3718,
 3741, 3742, 3759, 3760, 3813, 3814, 3863, 3864, 3865, 3866, 3871, 3872, 3875, 3876, 3883, 3884, 3913, 3914,
 3943, 3944, 3963, 3964, 3967, 3968, 3973, 3981, 3982, 3999, 4000, 4013, 4014, 4095, 4096, 4113, 4114, 4141,
 4142, 4159, 4160, 4175, 4176, 4181, 4182, 4183, 4184, 4195, 4196, 4219, 4220, 4223, 4224, 4233, 4234, 4261,
 4262, 4263, 4264, 4281, 4282, 4319, 4320, 4349, 4350, 4351, 4352, 4371, 4372, 4377, 4378, 4397, 4398, 4405,
 4406, 4407, 4408, 4421, 4422, 4425, 4426, 4455, 4456, 4577, 4578, 4631, 4632, 4641, 4642, 4645, 4646, 4657,
 4658, 4667, 4668, 4685, 4686 ,4757, 4758, 4787, 4788, 4805, 4806, 4881, 4882, 4897, 4898, 4905, 4906, 5009,
 5010, 5021, 5022, 5049, 5050, 5059, 5060, 5087, 5088, 5139, 5140, 5147, 5148, 5237, 5238, 5247, 5248, 5261,
 5262, 5273, 5274, 5283, 5284, 5297, 5298, 5353, 5354, 5355, 5356, 5379, 5380, 5405, 5406, 5429, 5430, 5431,
 5432, 5551, 5552, 5561, 5562, 5649, 5650, 5653, 5654, 5675, 5676, 5687, 5688, 5703, 5704, 5729, 5730, 5735,
 5736, 5741, 5742, 5749, 5750, 5819, 5820, 5823, 5824, 5833, 5834, 5895, 5896, 5945, 5946, 5985, 5986, 6077,
 6078, 6123, 6124, 6141, 6142, 6221, 6222, 6321, 6322, 6341, 6342, 6351, 6352, 6371, 6372, 6433, 6434, 6531,
 6532, 6535, 6536, 6545, 6546, 6553, 6554, 6677, 6678, 6679, 6680, 6681, 6682, 6683, 6684, 6693, 6694, 6759,
 6760, 6789, 6790, 6807, 6808, 6873, 6874, 6875, 6876, 6893, 6894, 6909, 6910, 6939, 6940, 6993, 6994, 7029,
 7030, 7037, 7038, 7087, 7088, 7103, 7104, 7121, 7122, 7123, 7124, 7129, 7130, 7251, 7252, 7271, 7272, 7277,
 7278, 7281, 7282]


for i in reversed(ex_list):
    data_dict['X_train'] = np.delete(data_dict['X_train'], i-1, axis=0)
    data_dict['y_train'] = np.delete(data_dict['y_train'], i-1, axis=0)

7282
7281
7278
7277
7272
7271
7252
7251
7130
7129
7124
7123
7122
7121
7104
7103
7088
7087
7038
7037
7030
7029
6994
6993
6940
6939
6910
6909
6894
6893
6876
6875
6874
6873
6808
6807
6790
6789
6760
6759
6694
6693
6684
6683
6682
6681
6680
6679
6678
6677
6554
6553
6546
6545
6536
6535
6532
6531
6434
6433
6372
6371
6352
6351
6342
6341
6322
6321
6222
6221
6142
6141
6124
6123
6078
6077
5986
5985
5946
5945
5896
5895
5834
5833
5824
5823
5820
5819
5750
5749
5742
5741
5736
5735
5730
5729
5704
5703
5688
5687
5676
5675
5654
5653
5650
5649
5562
5561
5552
5551
5432
5431
5430
5429
5406
5405
5380
5379
5356
5355
5354
5353
5298
5297
5284
5283
5274
5273
5262
5261
5248
5247
5238
5237
5148
5147
5140
5139
5088
5087
5060
5059
5050
5049
5022
5021
5010
5009
4906
4905
4898
4897
4882
4881
4806
4805
4788
4787
4758
4757
4686
4685
4668
4667
4658
4657
4646
4645
4642
4641
4632
4631
4578
4577
4456
4455
4426
4425
4422
4421
4408
4407
4406
4405
4398
4397
4378
4377
4372
4371
4352
4351
4350
4349
4320
4319
4282
4281
4264
4263


In [11]:
###### tf.logging.set_verbosity(tf.logging.INFO)
tf.reset_default_graph()
print(len(subject_list))

700


In [6]:
num_of_classes = len(subject_list)
def cnn_model_fn(features, labels, mode):
    """CNN model function"""
    
    input_layer = tf.reshape(features["x"], [-1, 96, 96, 1])
    conv1 = tf.layers.conv2d(inputs=input_layer,
                            filters=32,
                            kernel_size=7,
                            padding="same",
                            kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                            activation=tf.nn.leaky_relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=2, strides =2)
    conv2 = tf.layers.conv2d(inputs=pool1,
                            filters=32,
                            kernel_size=5,
                            padding="same",
                            kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                            activation=tf.nn.leaky_relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv1, pool_size=2, strides =2)
    conv3 = tf.layers.conv2d(inputs=pool2,
                            filters=32,
                            kernel_size=3,
                            padding="same",
                            kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                            activation=tf.nn.leaky_relu)
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=2, strides =2)
    pool3_flat = tf.reshape(pool3, [-1, 24 * 24 * 32])
    dense = tf.layers.dense(inputs=pool3_flat, units=24 * 24 * 32, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=0.1, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    logits = tf.layers.dense(inputs=dropout, units=num_of_classes)
    classes = tf.as_string(tf.argmax(input=logits, axis=1, name="class"))
    predictions = {
        "classes" : tf.argmax(input=logits, axis=1),
        "probabilities" : tf.nn.softmax(logits, name="softmax_tensor"),
        "dense" : dense,
        "pool3" : pool3_flat
        }
        
    if mode == tf.estimator.ModeKeys.PREDICT:
        export_outputs = {"predictions" : tf.estimator.export.PredictOutput(predictions)}
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions, export_outputs=export_outputs)
    
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth = num_of_classes)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        # This is wrong, need to fix
        metrics = {
            "accuracy" : tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
        }
        tf.summary.scalar("accuracy", metrics["accuracy"][1])
        merge_summary_op = tf.summary.merge_all
        lr = 1e-4
        step_rate = 10000
        decay = 0.8
        learning_rate = tf.train.exponential_decay(lr, global_step=tf.train.get_or_create_global_step(), decay_steps=step_rate, decay_rate=decay, staircase=True)
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    eval_metrics_op = {"accuracy" : tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metrics_op)

In [7]:
face_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="train/train_gs_2/")
tensors_to_log = {"probabilities" : "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=200)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'train/train_gs_2/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f316c9a9390>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [8]:
training_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x" : data_dict['X_train']},
                                                       y=data_dict['y_train'],
                                                       batch_size=25,
                                                       num_epochs=None,
                                                       shuffle=True)
face_classifier.train(input_fn=training_input_fn, steps=100, hooks=[logging_hook])

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use the retry module or similar alternatives.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train/train_gs_2/model.ckpt-24576
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 24577 into train/train_gs_2/model.ckpt.
INFO:tensorflow:probabilities = [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
INFO:tensorflow:loss = 4.768371e-09, step = 24576
INFO:tensorflow:Saving checkpoints for 24676 into train/train_gs_2/model.ckpt.
INFO:tensorflow:Loss for final step: 0.0.


In [9]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x" : data_dict['X_eval']},
                                                   y=data_dict['y_eval'],
                                                   num_epochs=1,
                                                   shuffle=False)
eval_results = face_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-26-13:41:49
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train/train_gs_2/model.ckpt-24576
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-26-13:41:51
INFO:tensorflow:Saving dict for global step 24576: accuracy = 0.75, global_step = 24576, loss = 2.3571055
{'accuracy': 0.75, 'loss': 2.3571055, 'global_step': 24576}


In [10]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x" : data_dict['X_train']},
                                                   y=data_dict['y_train'],
                                                   num_epochs=1,
                                                   shuffle=False)
eval_results = face_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-26-13:41:54
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train/train_gs_2/model.ckpt-24576
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-26-13:41:59
INFO:tensorflow:Saving dict for global step 24576: accuracy = 1.0, global_step = 24576, loss = 2.9124994e-09
{'accuracy': 1.0, 'loss': 2.9124994e-09, 'global_step': 24576}


In [23]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x" : data_dict['X_train'][:,:,:,:]}, num_epochs=1, shuffle = False)
predictions = list(face_classifier.predict(input_fn=predict_input_fn))
print(len(predictions))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train/train_gs_2/model.ckpt-24476
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
6931


In [24]:
count = 0
for j in range (data_dict['X_train'].shape[0]):
    lowest = 1000
    index = -1
    for i in range(data_dict['X_train'].shape[0]):
        if i == j:
            continue
        curr = np.mean((predictions[i]["dense"]- predictions[j]["dense"])**2)
        if curr < lowest:
            lowest = curr
            index = i
    
    if (data_dict['y_train'][j] == data_dict['y_train'][index]):
        print("SUCCSESS!")
        print(lowest)
        count += 1
    else:
        print("FAILED!")

print(count/data_dict['X_train'].shape[0])

SUCCSESS!
0.097379155
SUCCSESS!
0.097379155
SUCCSESS!
0.0806469
SUCCSESS!
0.0806469
SUCCSESS!
0.17841487
SUCCSESS!
0.17388146
SUCCSESS!
0.13861679
SUCCSESS!
0.13861679
SUCCSESS!
0.057841726
SUCCSESS!
0.06692606
SUCCSESS!
0.20977174
SUCCSESS!
0.20977174
SUCCSESS!
0.10981306
SUCCSESS!
0.11685855
FAILED!
FAILED!
SUCCSESS!
0.07372924
SUCCSESS!
0.07525635
SUCCSESS!
0.10844281
SUCCSESS!
0.11816761
SUCCSESS!
0.11233437
SUCCSESS!
0.10841355
FAILED!
FAILED!
SUCCSESS!
0.26000154
SUCCSESS!
0.25748357
SUCCSESS!
0.11233437
SUCCSESS!
0.10841355
SUCCSESS!
0.25748357
SUCCSESS!
0.27525738
SUCCSESS!
0.057841726
SUCCSESS!
0.06692606
SUCCSESS!
0.07372924
SUCCSESS!
0.07525635
SUCCSESS!
0.10981306
SUCCSESS!
0.11685855
SUCCSESS!
0.10844281
SUCCSESS!
0.11816761
SUCCSESS!
0.41777587
SUCCSESS!
0.41777587
SUCCSESS!
0.13156839
SUCCSESS!
0.12154195
SUCCSESS!
0.48898834
FAILED!
SUCCSESS!
0.3816662
SUCCSESS!
0.26605538
SUCCSESS!
0.16667315
SUCCSESS!
0.17344528
SUCCSESS!
0.16667315
SUCCSESS!
0.17344528
SUCCSESS!
0.23

SUCCSESS!
0.0670041
SUCCSESS!
0.45380464
SUCCSESS!
0.45380464
SUCCSESS!
0.3698996
SUCCSESS!
0.37383854
SUCCSESS!
0.08834372
SUCCSESS!
0.09580533
SUCCSESS!
0.45521933
SUCCSESS!
0.3698996
SUCCSESS!
0.08834372
SUCCSESS!
0.09580533
SUCCSESS!
0.09173721
SUCCSESS!
0.09173721
SUCCSESS!
0.13860522
SUCCSESS!
0.16892716
SUCCSESS!
0.16892716
SUCCSESS!
0.13860522
SUCCSESS!
0.04802412
SUCCSESS!
0.04802412
SUCCSESS!
0.042182367
SUCCSESS!
0.04869871
SUCCSESS!
0.30318773
SUCCSESS!
0.30318773
SUCCSESS!
0.36237258
SUCCSESS!
0.36237258
SUCCSESS!
0.042182367
SUCCSESS!
0.04869871
SUCCSESS!
0.1826321
SUCCSESS!
0.15481427
SUCCSESS!
0.1826321
SUCCSESS!
0.15481427
SUCCSESS!
0.32531443
SUCCSESS!
0.37359366
SUCCSESS!
0.29836932
SUCCSESS!
0.29836932
SUCCSESS!
0.43518338
SUCCSESS!
0.7967606
SUCCSESS!
0.4775416
SUCCSESS!
0.4775416
SUCCSESS!
0.7096613
SUCCSESS!
0.75819886
SUCCSESS!
0.43518338
SUCCSESS!
0.6196904
SUCCSESS!
0.2210676
SUCCSESS!
0.23648395
SUCCSESS!
0.2210676
SUCCSESS!
0.23648395
SUCCSESS!
0.51760924
SU

SUCCSESS!
0.2240152
FAILED!
FAILED!
FAILED!
FAILED!
SUCCSESS!
0.053737536
SUCCSESS!
0.054406628
SUCCSESS!
0.43139666
SUCCSESS!
0.5398147
SUCCSESS!
0.1637168
SUCCSESS!
0.20039111
SUCCSESS!
0.055281702
SUCCSESS!
0.055281702
SUCCSESS!
0.72236454
FAILED!
SUCCSESS!
0.27434063
SUCCSESS!
0.27434063
SUCCSESS!
0.21192123
SUCCSESS!
0.23944725
SUCCSESS!
0.1637168
SUCCSESS!
0.20039111
SUCCSESS!
0.29402438
SUCCSESS!
0.29402438
SUCCSESS!
0.34909037
SUCCSESS!
0.34909037
SUCCSESS!
0.1418728
SUCCSESS!
0.1418728
SUCCSESS!
0.19449446
SUCCSESS!
0.21217121
SUCCSESS!
0.35298744
SUCCSESS!
0.35298744
SUCCSESS!
0.37298048
SUCCSESS!
0.36437395
SUCCSESS!
0.12954512
SUCCSESS!
0.12954512
FAILED!
FAILED!
SUCCSESS!
0.34240022
SUCCSESS!
0.31649476
SUCCSESS!
0.27869508
SUCCSESS!
0.27869508
SUCCSESS!
0.17652592
SUCCSESS!
0.17652592
SUCCSESS!
0.15375622
SUCCSESS!
0.15375622
SUCCSESS!
0.29359898
SUCCSESS!
0.29480895
SUCCSESS!
0.15210073
SUCCSESS!
0.15210073
SUCCSESS!
0.5701301
SUCCSESS!
0.5701301
FAILED!
SUCCSESS!
0.5566

SUCCSESS!
0.3706953
SUCCSESS!
0.3706953
SUCCSESS!
0.07835955
SUCCSESS!
0.07885629
SUCCSESS!
0.07835955
SUCCSESS!
0.0794679
SUCCSESS!
0.06462308
SUCCSESS!
0.06462308
SUCCSESS!
0.23300579
SUCCSESS!
0.25060976
SUCCSESS!
0.23300579
SUCCSESS!
0.24026436
FAILED!
FAILED!
SUCCSESS!
0.08846972
SUCCSESS!
0.08846972
SUCCSESS!
0.035693996
SUCCSESS!
0.038079504
SUCCSESS!
0.17194073
SUCCSESS!
0.17194073
SUCCSESS!
0.035693996
SUCCSESS!
0.038079504
SUCCSESS!
0.058326773
SUCCSESS!
0.0600964
SUCCSESS!
0.058326773
SUCCSESS!
0.0600964
SUCCSESS!
0.076227754
SUCCSESS!
0.07539427
SUCCSESS!
0.3390572
SUCCSESS!
0.3390572
SUCCSESS!
0.20830913
SUCCSESS!
0.20830913
SUCCSESS!
0.076227754
SUCCSESS!
0.07539427
SUCCSESS!
0.05496197
SUCCSESS!
0.053717744
FAILED!
FAILED!
FAILED!
FAILED!
FAILED!
FAILED!
SUCCSESS!
0.061112802
SUCCSESS!
0.061112802
SUCCSESS!
0.07814007
SUCCSESS!
0.07814007
SUCCSESS!
0.05496197
SUCCSESS!
0.053717744
SUCCSESS!
0.21248052
SUCCSESS!
0.21379818
SUCCSESS!
0.1551401
SUCCSESS!
0.13301943
SUCCSESS

SUCCSESS!
0.0016128441
SUCCSESS!
0.002102043
SUCCSESS!
0.0016128441
FAILED!
FAILED!
SUCCSESS!
0.29907525
SUCCSESS!
0.24644798
SUCCSESS!
0.082477085
SUCCSESS!
0.082477085
FAILED!
FAILED!
FAILED!
FAILED!
FAILED!
FAILED!
SUCCSESS!
0.025121456
SUCCSESS!
0.027129369
SUCCSESS!
0.025121456
SUCCSESS!
0.027129369
FAILED!
FAILED!
SUCCSESS!
0.40918797
SUCCSESS!
0.43966544
FAILED!
FAILED!
FAILED!
FAILED!
FAILED!
FAILED!
FAILED!
FAILED!
SUCCSESS!
0.31925505
SUCCSESS!
0.31925505
FAILED!
FAILED!
SUCCSESS!
0.31667575
SUCCSESS!
0.31667575
SUCCSESS!
0.7774725
SUCCSESS!
0.837386
SUCCSESS!
0.023396708
SUCCSESS!
0.023396708
SUCCSESS!
0.021482078
SUCCSESS!
0.021482078
SUCCSESS!
0.13339925
SUCCSESS!
0.13339925
SUCCSESS!
0.11469621
SUCCSESS!
0.11469621
SUCCSESS!
0.03377423
SUCCSESS!
0.03601027
SUCCSESS!
0.03377423
SUCCSESS!
0.03601027
FAILED!
FAILED!
FAILED!
FAILED!
SUCCSESS!
0.050892726
SUCCSESS!
0.046936244
FAILED!
FAILED!
SUCCSESS!
0.050892726
SUCCSESS!
0.046936244
FAILED!
FAILED!
SUCCSESS!
0.08686817
SUCC

...

SUCCSESS!
0.07081376
SUCCSESS!
0.24194254
FAILED!
SUCCSESS!
0.17535816
SUCCSESS!
0.17535816
SUCCSESS!
0.28248993
SUCCSESS!
0.28248993
SUCCSESS!
0.15352878
SUCCSESS!
0.15352878
SUCCSESS!
0.32136604
SUCCSESS!
0.32572585
SUCCSESS!
0.32136604
SUCCSESS!
0.32572585
SUCCSESS!
0.11259772
SUCCSESS!
0.11259772
SUCCSESS!
0.13939406
SUCCSESS!
0.13939406
SUCCSESS!
0.11743193
SUCCSESS!
0.11743193
SUCCSESS!
0.121839814
SUCCSESS!
0.121839814
SUCCSESS!
0.12758632
SUCCSESS!
0.12758632
SUCCSESS!
0.115793586
SUCCSESS!
0.115793586
SUCCSESS!
0.13594958
SUCCSESS!
0.13035533
SUCCSESS!
0.27333376
SUCCSESS!
0.27333376
SUCCSESS!
0.053551123
SUCCSESS!
0.053551123
SUCCSESS!
0.18295278
SUCCSESS!
0.18295278
SUCCSESS!
0.03882369
SUCCSESS!
0.038868964
FAILED!
FAILED!
SUCCSESS!
0.030043973
SUCCSESS!
0.027907431
SUCCSESS!
0.03882369
SUCCSESS!
0.038868964
SUCCSESS!
0.030043973
SUCCSESS!
0.027907431
FAILED!
FAILED!
FAILED!
FAILED!
SUCCSESS!
0.17197308
SUCCSESS!
0.17197308
FAILED!
SUCCSESS!
0.24233812
SUCCSESS!
0.047152516

SUCCSESS!
0.19909607
SUCCSESS!
0.1979661
SUCCSESS!
0.20625746
SUCCSESS!
0.20962258
SUCCSESS!
0.27716854
SUCCSESS!
0.27117056
SUCCSESS!
0.2744568
SUCCSESS!
0.27117056
SUCCSESS!
0.22240697
SUCCSESS!
0.26515803
SUCCSESS!
0.20625746
SUCCSESS!
0.20962258
SUCCSESS!
0.19909607
SUCCSESS!
0.1979661
SUCCSESS!
0.2748405
SUCCSESS!
0.2744568
SUCCSESS!
0.32672614
SUCCSESS!
0.32672614
SUCCSESS!
0.25875252
SUCCSESS!
0.27130383
SUCCSESS!
0.2380583
SUCCSESS!
0.25048557
FAILED!
FAILED!
FAILED!
FAILED!
SUCCSESS!
0.25048557
SUCCSESS!
0.2380583
SUCCSESS!
0.06674118
SUCCSESS!
0.06674118
FAILED!
FAILED!
SUCCSESS!
0.18473327
SUCCSESS!
0.18473327
SUCCSESS!
0.30245227
FAILED!
FAILED!
SUCCSESS!
0.2708856
FAILED!
FAILED!
SUCCSESS!
0.18036875
SUCCSESS!
0.18831025
SUCCSESS!
0.18036875
SUCCSESS!
0.18831025
SUCCSESS!
0.25366795
SUCCSESS!
0.25366795
SUCCSESS!
0.23895568
SUCCSESS!
0.23895568
SUCCSESS!
0.2528856
FAILED!
SUCCSESS!
0.17417017
SUCCSESS!
0.17417017
SUCCSESS!
0.07516578
SUCCSESS!
0.07516578
SUCCSESS!
0.216897

SUCCSESS!
0.20367742
SUCCSESS!
0.19442369
SUCCSESS!
0.19442369
SUCCSESS!
0.02129878
SUCCSESS!
0.02129878
SUCCSESS!
0.15458599
SUCCSESS!
0.15458599
SUCCSESS!
0.2143102
SUCCSESS!
0.2143102
SUCCSESS!
0.2429832
SUCCSESS!
0.24570152
SUCCSESS!
0.3726442
SUCCSESS!
0.4249073
FAILED!
FAILED!
SUCCSESS!
0.18684088
SUCCSESS!
0.2055183
FAILED!
FAILED!
SUCCSESS!
0.1625458
SUCCSESS!
0.1625458
SUCCSESS!
0.14945875
SUCCSESS!
0.14945875
SUCCSESS!
0.08292749
SUCCSESS!
0.08053589
SUCCSESS!
0.08292749
SUCCSESS!
0.08053589
SUCCSESS!
0.22356918
SUCCSESS!
0.22356918
SUCCSESS!
0.16421236
SUCCSESS!
0.15930922
SUCCSESS!
0.104128055
SUCCSESS!
0.104128055
SUCCSESS!
0.08922806
SUCCSESS!
0.094197586
SUCCSESS!
0.06283954
SUCCSESS!
0.06283954
FAILED!
FAILED!
SUCCSESS!
0.15930922
SUCCSESS!
0.16421236
SUCCSESS!
0.08922806
SUCCSESS!
0.094197586
SUCCSESS!
0.31693453
SUCCSESS!
0.31693453
SUCCSESS!
0.22468631
SUCCSESS!
0.2541654
SUCCSESS!
0.20478171
SUCCSESS!
0.19508888
SUCCSESS!
0.033199675
SUCCSESS!
0.033199675
SUCCSESS!


SUCCSESS!
0.36980817
SUCCSESS!
0.36980817
SUCCSESS!
0.14654575
SUCCSESS!
0.14654575
FAILED!
FAILED!
SUCCSESS!
0.266645
FAILED!
SUCCSESS!
0.21778876
SUCCSESS!
0.19584186
SUCCSESS!
0.28218967
SUCCSESS!
0.266645
SUCCSESS!
0.27073494
SUCCSESS!
0.26868784
FAILED!
FAILED!
SUCCSESS!
0.1896687
SUCCSESS!
0.1896687
SUCCSESS!
0.13797326
SUCCSESS!
0.13797326
SUCCSESS!
0.12542915
SUCCSESS!
0.12542915
SUCCSESS!
0.3639236
SUCCSESS!
0.37486425
FAILED!
FAILED!
SUCCSESS!
0.13932869
SUCCSESS!
0.13932869
SUCCSESS!
0.30854544
SUCCSESS!
0.30854544
SUCCSESS!
0.1576462
SUCCSESS!
0.12987036
SUCCSESS!
0.20845042
SUCCSESS!
0.2215845
SUCCSESS!
0.29622558
SUCCSESS!
0.29622558
SUCCSESS!
0.1576462
SUCCSESS!
0.12987036
SUCCSESS!
0.058706336
SUCCSESS!
0.058706336
SUCCSESS!
0.20475852
SUCCSESS!
0.19299941
SUCCSESS!
0.20845042
SUCCSESS!
0.2215845
SUCCSESS!
0.10212594
SUCCSESS!
0.111322366
SUCCSESS!
0.24043295
SUCCSESS!
0.262965
SUCCSESS!
0.10212594
SUCCSESS!
0.111322366
SUCCSESS!
0.12488857
SUCCSESS!
0.11773678
SUCCSESS

SUCCSESS!
0.057105362
SUCCSESS!
0.214181
SUCCSESS!
0.23075785
SUCCSESS!
0.22687371
SUCCSESS!
0.22687371
SUCCSESS!
0.4137694
FAILED!
SUCCSESS!
0.27957603
SUCCSESS!
0.26394027
SUCCSESS!
0.22350508
SUCCSESS!
0.23075785
SUCCSESS!
0.18750626
SUCCSESS!
0.17429763
SUCCSESS!
0.18750626
SUCCSESS!
0.17429763
SUCCSESS!
0.1131351
SUCCSESS!
0.10936292
SUCCSESS!
0.056664243
SUCCSESS!
0.057105362
SUCCSESS!
0.29652774
SUCCSESS!
0.2769139
SUCCSESS!
0.096514285
SUCCSESS!
0.096514285
SUCCSESS!
0.06977349
SUCCSESS!
0.06414601
SUCCSESS!
0.030667812
SUCCSESS!
0.030667812
SUCCSESS!
0.07958613
SUCCSESS!
0.084036745
SUCCSESS!
0.069149576
SUCCSESS!
0.074164495
SUCCSESS!
0.06414601
SUCCSESS!
0.06977349
SUCCSESS!
0.096949965
SUCCSESS!
0.096949965
SUCCSESS!
0.22434007
SUCCSESS!
0.22434007
SUCCSESS!
0.07958613
SUCCSESS!
0.084036745
SUCCSESS!
0.035945315
SUCCSESS!
0.035945315
SUCCSESS!
0.057577245
SUCCSESS!
0.057577245
SUCCSESS!
0.17817877
SUCCSESS!
0.17817877
SUCCSESS!
0.12239557
SUCCSESS!
0.12239557
SUCCSESS!
0.08

In [18]:
with open('known_faces/faces.pickle', 'rb') as handle:
    known_pics_dict = pickle.load(handle)

dense_dict = {}
    
for file, pic in known_pics_dict.items():
    pic = (pic - mean_image)/std_image
    predict_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x" : pic}, num_epochs=1, shuffle = False)
    prediction = list(face_classifier.predict(input_fn=predict_input_fn))[0]
    
    dense_dict[file] = prediction["dense"]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train/train_gs_2/model.ckpt-14476
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train/train_gs_2/model.ckpt-14476
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train/train_gs_2/model.ckpt-14476
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train/train_gs_2/model.ckpt-14476
INFO:te

In [19]:
for file, dense in dense_dict.items():
    lowest = 1000
    key_file = ''
    for key in dense_dict.keys():
        if key == file:
            continue
        curr = np.mean((dense - dense_dict[key])**2)
        if curr < lowest:
            lowest = curr
            key_file = key
    print("Original is {} and closest is {} with score of {}".format(file, key_file, lowest))

Original is shahar4.jpg and closest is michi2.jpg with score of 0.24735122919082642
Original is michi4.jpg and closest is shahar4.jpg with score of 0.5207279324531555
Original is shahar1.jpg and closest is shahar2.jpg with score of 0.6190715432167053
Original is shahar2.jpg and closest is shahar5.jpg with score of 0.42375075817108154
Original is michi5.jpg and closest is michi3.jpg with score of 0.6107949614524841
Original is michi2.jpg and closest is michi3.jpg with score of 0.15404987335205078
Original is shahar5.jpg and closest is shahar2.jpg with score of 0.42375075817108154
Original is michi3.jpg and closest is michi2.jpg with score of 0.15404987335205078


In [9]:
def serving_input_receiver_fn():
    """An input receiver that expects a serialized tf.Example."""
    serialized_tf_example = tf.placeholder(dtype=tf.string, shape=[None], name='input_tensors')
    receiver_tensors = {"predictor_inputs" : serialized_tf_example}
    feature_spec = {"x" : tf.FixedLenFeature([96, 96 ,1], tf.float32)}
    features = tf.parse_example(serialized_tf_example, feature_spec)
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)

exported_model = face_classifier.export_savedmodel("trained/trained_0", serving_input_receiver_fn=serving_input_receiver_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predictions', 'serving_default']
INFO:tensorflow:Restoring parameters from train/train_gs_2/model.ckpt-24676
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b"trained/trained_0/temp-b'1532617597'/saved_model.pb"


In [15]:
print(mean_image, std_image)

103.659775 52.517044
